# Representación semántica y búsqueda
Autor: Eric S. Tellez <eric.tellez@infotec.mx>

## Usando la nube de puntos

La representación de nube de puntos puede usarse directamente sin realizar prototipos, sin embargo, es posible que dada su complejidad, dimensión intrinseca, tenga poca utilidad en grandes volumenes de información. Por ejemplo, es posible utilizar la distancia de [Hausdorff](https://en.wikipedia.org/wiki/Hausdorff_distance); que entre dos nubes de puntos (documentos para este caso) esta definida como sigue:

$$ H_d(U, V) = \max \left\{ \max_{u \in U} \min_{x \in V} d(u, x), \max_{v \in V} \min_{x \in U} d(v, x) \right\} $$

Donde $\textsf{nn}$ encuentra el vecino cercano del primer argumento en el segundo. Podemos ver la intuición de esta distancia, analizando las partes de la expresión. 
La diferencia entre ambos conjuntos se representa con el máximo de las distancias mínimas. Note que esta parte es idéntica a resolver una búsqueda de vecinos cercanos. La parte más externa de la expresión se repite para los dos conjuntos para preservar la simétria de la función.

De la misma forma, es posible obtener variaciones de interés como considerar todas las distancias cercanas en lugar de solo las máximas, lo cual puede reducir formas caprichosas de _outlayers_.

$$ H^+_d(U, V) = \frac{1}{2|U|} \sum_{u \in U} \min_{x \in V} d(u, x) + \frac{1}{2|V|}\sum_{v \in V} \min_{x \in U} d(v, x) $$

La información local de una representación de bolsa de palabras también puede ser aprovechada, por ejemplo, añadiendo información de $\textsf{tp}$

$$ H^\textsf{pt}_d(U, V) = \max \left\{
        \max_{u \in U} \left\{\textsf{pt}(u, U) \min_{x \in V} d(u, x)\right\},
        \max_{v \in V} \left\{\textsf{pt}(v, V) \min_{x \in U} d(v, x)\right\}
     \right\} $$

# Notas adicionales
La versión de $H^\textsf{pt}_d$ que usa la probabilidad de término $\textsf{pt}$ es similar a la aproximación 2 listada en [@KSKW2015]. Los prototipos son similares a la aproximación 1 del mismo artículo. En el artículo se presenta la función de distancia _word mover's distance_ (WMD) que es una adecuación de la Earths mover's distance (EMD) [PW2009]. EMD es la solución óptima a un problema de transportación, la WMD es la adaptación teniendo en cuenta embeddings de palabras.

# Ejemplo


In [1]:
using Pkg
Pkg.activate(".")
!isfile("Manifest.toml") && Pkg.add([
    PackageSpec(name="SimilaritySearch", version="0.9"),
    PackageSpec(name="TextSearch", version="0.13"),
    PackageSpec(name="Plots"),
    PackageSpec(name="KNearestCenters"),
    PackageSpec(name="HypertextLiteral"),
    PackageSpec(name="CSV"),
    PackageSpec(name="DataFrames"),
    PackageSpec(name="Word2Vec"),
    PackageSpec(name="Embeddings"),
])

using SimilaritySearch, TextSearch, Plots, Word2Vec, KNearestCenters, LinearAlgebra, HypertextLiteral, CSV, DataFrames, Base64, Random
using Embeddings: load_embeddings, FastText_Text
using Downloads: download

  Activating project at `~/IR-2022/Unidades`


In [2]:
]status

      Status `~/IR-2022/Unidades/Project.toml`
  [944b1d66] CodecZlib v0.7.0
  [ac1192a8] HypertextLiteral v0.9.4
  [b20bd276] InvertedFiles v0.4.2 `../../Research/InvertedFiles.jl`
  [682c06a0] JSON v0.21.3
  [4dca28ae] KNearestCenters v0.7.1
  [8ef0a80b] Languages v0.4.3
  [eb30cadb] MLDatasets v0.7.3
  [053f045d] SimilaritySearch v0.9.4 `../../Research/SimilaritySearch.jl`
  [fb8f903a] Snowball v0.1.0
  [2913bbd2] StatsBase v0.33.18
  [7f6f6c8a] TextSearch v0.13.0 `../../Research/TextSearch.jl`
  [c64b6f0f] Word2Vec v0.5.3


In [3]:
function word2vec_embeddings(textconfig, corpus, embfile; dim=32, w2vtmp=tempfile(pwd()))
    isfile(embfile) && return
    tcorpus = tokenize_corpus(textconfig, corpus)
    voc = Vocabulary(textconfig, tcorpus)
    fvoc = filter_tokens(voc) do t
        5 <= t.ndocs <= 2000
    end
    valid = Set(fvoc.token)

    open(w2vtmp, "w") do f
        for tokens in tcorpus
            tokens = filter!(t -> t in valid, tokens)
            if length(tokens) > 7 # minimum number of tokens in a message
                println(f, join(tokens, ' '))
            end
        end
    end

    word2vec(w2vtmp, embfile, size=dim, iter=15, threads=Threads.nthreads()-1)
end


word2vec_embeddings (generic function with 1 method)

In [4]:
textconfig = TextConfig(; nlist=[1], qlist=[], slist=[], lc=true,
    group_usr=true, group_url=true, group_num=true, del_diac=true, del_punc=true)

filename = "../data/spanish-news-in-twitter.csv"
embfile = filename * ".w2v.vec"
@time D = CSV.read(filename, DataFrame; delim=',', missingstring="XXXX")
@time D[:, :text] = String.(base64decode.(D.text))
rm(embfile)
word2vec_embeddings(textconfig, D.text, embfile; dim=64, w2vtmp=joinpath(pwd(), "w2v.tmp"))
E = load_embeddings(FastText_Text, embfile)


 12.754743 seconds (19.95 M allocations: 1.540 GiB, 4.16% gc time, 99.52% compilation time)
  0.411904 seconds (1.72 M allocations: 125.889 MiB, 8.06% gc time, 71.98% compilation time)
Starting training using file /home/sadit/IR-2022/Unidades/w2v.tmp
Vocab size: 12135
Words in train file: 442523


Embeddings.EmbeddingTable{Matrix{Float32}, Vector{String}}(Float32[0.006254 -0.001177 … -0.005999 0.006102; 0.006905 0.001611 … 0.007287 0.005898; … ; -0.003277 0.001377 … 0.001926 -0.005139; 0.003372 0.002838 … 0.000214 0.003586], ["</s>", "ha", "sus", "presidente", "contra", "anos", "fue", "tras", "pero", "gobierno"  …  "#ondasdelpodcast", "🤢", "#premiosortegaygasset", "infiel", "indigenista", "contenedores", "#cerocahui", "chueco", "yrma", "lydya"])

In [5]:
T = let corpus=D.text
    voc = Vocabulary(textconfig, corpus; thesaurus=E.vocab)
    model = VectorModel(BinaryGlobalWeighting(), TpWeighting(), voc)
    (; textconfig, model, voc)
end

(textconfig = TextConfig(true, false, true, true, true, true, false, true, Int8[], Int8[1], Skipgram[], IdentityTokenTransformation()), model = {VectorModel global_weighting=BinaryGlobalWeighting(), local_weighting=TpWeighting(), train-voc=12135, train-n=41999, maxoccs=1987}, voc = Vocabulary(["</s>", "ha", "sus", "presidente", "contra", "anos", "fue", "tras", "pero", "gobierno"  …  "#ondasdelpodcast", "🤢", "#premiosortegaygasset", "infiel", "indigenista", "contenedores", "#cerocahui", "chueco", "yrma", "lydya"], Int32[0, 1987, 1953, 1926, 1954, 1956, 1614, 1411, 1361, 1359  …  5, 6, 5, 5, 6, 6, 5, 5, 6, 6], Int32[0, 1987, 1953, 1926, 1954, 1956, 1614, 1411, 1361, 1359  …  5, 6, 5, 5, 6, 6, 5, 5, 6, 6], Dict{String, UInt32}("focos" => 0x0000225f, "#videos" => 0x00001754, "libertad" => 0x000001be, "ayudas" => 0x00000f6f, "henry" => 0x00001fb4, "islamico" => 0x0000163b, "directivos" => 0x00001625, "soporte" => 0x000013bb, "astronautas" => 0x00002584, "advirtieron" => 0x0000234f…), 41999)

In [6]:
cloud(text, E, T) = vectorize(T.model, T.textconfig, text; normalize=false)
cloud_corpus(corpus, E, T) = vectorize_corpus(T.model, T.textconfig, corpus; normalize=false)

cloud_corpus (generic function with 1 method)

In [7]:
struct AWMD <: SemiMetric
    X::MatrixDatabase
    results::Vector{KnnResult}
    dcos::NormalizedCosineDistance
end

AWMD(X) = AWMD(X, [KnnResult(1) for _ in 1:Threads.nthreads()], NormalizedCosineDistance())

const PI_2 = convert(Float32, pi / 2)


@inline function nn_(X, res, dcos, u, V)
    res = reuse!(res)

    for (vid, _) in V
        push!(res, vid, evaluate(dcos, u, X[vid]))
    end
    
    minimum(res)
end

function onesidewmd(X, res, dcos, U, V)
    d::Float32 = zero(Float32)
    @fastmath for (uid, w) in U
        if !haskey(V, uid) # evaluates to zero
            d = max(d, w * nn_(X, res, dcos, X[uid], V))
        end
    end
    
    d
end

function SimilaritySearch.evaluate(wmd::AWMD, U::T, V::T) where T
    length(U) == 1 && haskey(U, 0) && return PI_2
    length(V) == 1 && haskey(V, 0) && return PI_2
    res = wmd.results[Threads.threadid()]
    dcos = wmd.dcos

    max(onesidewmd(wmd.X, res, dcos, U, V), onesidewmd(wmd.X, res, dcos, V, U))
end


In [8]:
db = VectorDatabase(cloud_corpus(D.text, E, T))
for c in eachcol(E.embeddings)
    normalize!(c)
end

dist = AWMD(MatrixDatabase(E.embeddings))

AWMD(MatrixDatabase{Matrix{Float32}}(Float32[0.17953031 -0.033541206 … -0.16976508 0.16736586; 0.19821823 0.045908995 … 0.20621407 0.16177055; … ; -0.09407112 0.039240647 … 0.05450368 -0.14095266; 0.09679824 0.08087506 … 0.006055964 0.09835693]), KnnResult[KnnResult(Int32[], Float32[], 1)], NormalizedCosineDistance())

In [9]:
#=recall = 0.8
index = SearchGraph(; dist, db, verbose=true)
#index!(index; callbacks=SearchGraphCallbacks(MinRecall(recall)))
index!(index)
optimize!(index, MinRecall(recall))
=#

In [10]:
index = ParallelExhaustiveSearch(; db, dist);

In [11]:
Threads.nthreads()

1

In [12]:
#@time res, _ = search(ex, db[111], KnnResult(10))
#=@time res, _ = search(ex, cloud("covid en México", E, T), KnnResult(10))

for (id_, dist_) in res
    display(@htl """<div style="padding: 0.5em;">
        <span style="width: 25%; margin: 0.25em;"> $(id_ => round(dist_; digits=3))</span>
        <span style="background-color: rgb(120, 60, 60); margin: 0.2em;">$(D.text[id_])</span>
    </div>""")
end
=#

### Búsqueda y presentación de los resultados

In [13]:
function search_and_display(index, qtext, k, D, E, T)
    res = KnnResult(k)
    q = cloud(qtext, E, T)
    @time search(index, q, res)
    
    L = []
    for (j, (id, d)) in enumerate(res)
        push!(L, @htl "<tr><td>$j</td><td>$id</td><td>$(round(d, digits=3))</td> <td>$(D.screen_name[id])</td><td> $(D.text[id])</td> </tr>")
    end

    display(@htl """<h2>resultados para "$qtext"</h2>
    <table>
    <th>  <td>id</td> <td>dist</td> <td>user</td> </td>message<td> </th>
        $L
    </table>
    """)
end


search_and_display (generic function with 1 method)

In [14]:

display(@htl "<h1>Ejemplos de búsqueda</h1>")
search_and_display(index, "el gobierno de andres manuel lopez", 7, D, E, T)
search_and_display(index, "trafico de drogas", 7, D, E, T)
search_and_display(index, "calentamiento global", 7, D, E, T)
search_and_display(index, "covid corona virus", 7, D, E, T)


<h1>Ejemplos de búsqueda</h1>

  0.982184 seconds (1.98 M allocations: 196.451 MiB, 7.43% gc time, 70.39% compilation time)


1,30139,0.06,EFEnoticias,"El presidente de México, Andrés Manuel López Obrador, celebró la aprobación de la reforma que permite a los funcionarios públicos manifestarse sobre el proceso de consulta de revocación de mandato, a celebrarse el 10 de abril. https://t.co/CR4WSfihEO"
2,16524,0.072,azucenau,"La jefa de gobierno de la Ciudad de México, Claudia Sheinbaum, aseguró que ella y el presidente de la República, Andrés Manuel López Obrador, “somos parte del mismo movimiento”, y que jamás será líder de oposición, pues “somos parte de la Cuarta Transformación”. https://t.co/FYoh0GuROr"
3,41283,0.087,Reforma,"El Presidente Andrés Manuel López Obrador propuso al Gobierno de #NuevoLeon acordar con empresarios que produzcan menos, usen menos #agua, y que así vaya más líquido a la gente. https://t.co/9kW1cPZBmj"
4,4111,0.101,PedroFerriz,RT @CentralFM_: #CentralFmEquilibrio Estados Unidos urgió al Gobierno de Andrés Manuel López Obrador a reanudar la autorización de import…
5,9801,0.133,JoseCardenas1,RT @JoseCardenas1: Sigue minuto a minuto el 3er informe de gobierno del presidente Andrés Manuel López Obrador (@lopezobrador_ )👉 https://t…
6,20826,0.164,El_Universal_Mx,RT @ElUniversal_SLP: #Entérate El gobernador Ricardo Gallardo Cardona pidió al presidente Andrés Manuel López Obrador apoyo para reforzar l…
7,35523,0.164,JoseCardenas1,"El Presidente Andrés Manuel López Obrador arribó esta tarde al Aeropuerto de La Habana, Cuba, para comenzar su gira de trabajo y sostener encuentros con el mandatario de la isla, Miguel Díaz-Canel https://t.co/GiBHboPAIG"


  0.200157 seconds (166.64 k allocations: 3.814 MiB)


1,1361,0.041,epigmenioibarra,"29 JoseE 30 JoseLuisL 31 Jhosivani 32 JulioCesarL El GIEI solicitó investigar el tráfico de drogas escondidas en autobuses como probable motivo del ataque a los normalistas, ya que un quinto autobús que apareció quemado fue omitido en la investigación oficial. #Ayotzinapa80Meses https://t.co/jbAcSdIuPI"
2,2113,0.041,epigmenioibarra,"29 JoseE 30 JoseLuisL 31 Jhosivani 32 JulioCesarL El GIEI solicitó investigar el tráfico de drogas escondidas en autobuses como probable motivo del ataque a los normalistas, ya que un quinto autobús que apareció quemado fue omitido en la investigación oficial. #Ayotzinapa80Meses https://t.co/BNAdU3bp0n"
3,3780,0.041,epigmenioibarra,"29 JoseE 30 JoseLuisL 31 Jhosivani 32 JulioCesarL El GIEI solicitó investigar el tráfico de drogas escondidas en autobuses como probable motivo del ataque a los normalistas, ya que un quinto autobús que apareció quemado fue omitido en la investigación oficial. #Ayotzinapa80Meses https://t.co/cR2d88sSeF"
4,4164,0.041,epigmenioibarra,"29 JoseE 30 JoseLuisL 31 Jhosivani 32 JulioCesarL El GIEI solicitó investigar el tráfico de drogas escondidas en autobuses como probable motivo del ataque a los normalistas, ya que un quinto autobús que apareció quemado fue omitido en la investigación oficial. #Ayotzinapa81Meses https://t.co/jOhSMS35Z5"
5,5391,0.041,epigmenioibarra,"29 JoseE 30 JoseLuisL 31 Jhosivani 32 JulioCesarL El GIEI solicitó investigar el tráfico de drogas escondidas en autobuses como probable motivo del ataque a los normalistas, ya que un quinto autobús que apareció quemado fue omitido en la investigación oficial. #Ayotzinapa81Meses https://t.co/cxi6tlCTMA"
6,5645,0.041,epigmenioibarra,"29 JoseE 30 JoseLuisL 31 Jhosivani 32 JulioCesarL El GIEI solicitó investigar el tráfico de drogas escondidas en autobuses como probable motivo del ataque a los normalistas, ya que un quinto autobús que apareció quemado fue omitido en la investigación oficial. #Ayotzinapa81Meses https://t.co/V6gw7dw6ZX"
7,18648,0.041,epigmenioibarra,"29 JoseE 30 JoseLuisL 31 Jhosivani 32 JulioCesarL El GIEI solicitó investigar el tráfico de drogas escondidas en autobuses como probable motivo del ataque a los normalistas, ya que un quinto autobús que apareció quemado fue omitido en la investigación oficial. #Ayotzinapa85meses https://t.co/KH0xDLkhWz"


  0.224001 seconds (166.64 k allocations: 3.814 MiB)


1,17410,0.047,TelemundoNews,"El acuerdo incluye promesas clave, entre ellas la de reducir la energía del carbón y las subvenciones a los combustibles fósiles, pero muchos críticos dicen que no ofrece suficiente apoyo a las naciones en desarrollo afectadas por el calentamiento global. https://t.co/foBCgxRNVT"
2,7442,0.051,patriciajaniot,"El calentamiento global es más rápido y grave de lo pensado. Unos diez años antes de lo estimado, se puede alcanzar el umbral de un incremento de la temperatura en 34.7 °F (1.5 ºC), con riesgos de desastres ""sin precedentes"" para la humanidad https://t.co/iQVOG3eVyZ"
3,10955,0.051,EFEnoticias,"El papa Francisco llegó hoy a Budapest, donde celebrará este domingo la misa de clausura del Congreso Eucarístico Internacional y se reunirá con los líderes del país centroeuropeo, para después continuar su visita de tres días a Eslovaquia. https://t.co/SE0e5IK4h7"
4,11613,0.061,EFEnoticias,"El Reino Unido presionó al resto de Gobiernos para que anuncien más medidas contra el calentamiento global, apenas cinco semanas antes de la próxima cumbre del clima (COP26) que acogerá su país. https://t.co/bmKjR0hXRm"
5,10963,0.063,SinEmbargoMX,"#Opinión | Ana Ortega de @THL_Latino ""El consumo de carne o lo que no se habla del calentamiento global"" Si no adoptamos una postura más firme ante el calentamiento global, el panorama de México está a punto de ponerse sombrío https://t.co/HL8v7EBp2x https://t.co/k8Az8hDr5y"
6,7766,0.073,CNNEE,"▶ Todavía podemos mitigar el daño al planeta por el calentamiento global si exigimos políticas públicas a gobiernos, dicen expertas. https://t.co/7eoc24sboY"
7,7410,0.103,AFPespanol,"#ÚLTIMAHORA El calentamiento global llegará a +1,5 °C hacia 2030, 10 años antes de lo previsto (ONU) #AFP https://t.co/KJT1ESxU2S"


  0.203944 seconds (166.64 k allocations: 3.814 MiB)


1,18569,0.051,EFEnoticias,"La Comisión Europea (CE) propuso este jueves limitar a 9 meses la validez del certificado digital covid y seguir respetando las medidas destinadas a frenar la propagación del virus, incluido el uso de las mascarillas, ante el aumento de los contagios. https://t.co/RswfZ609kt"
2,25138,0.051,galvanochoa,Debe ser duro admitir que el primer gobernador de Morena es un infractor. Pasó por encima de la ley para intentar quedarse en el gobierno más allá de los dos años para los que fue elegido y está relacionado con el asesinato de Lourdes Maldonado 👉🏼 https://t.co/cNrm71fiUF https://t.co/VilyLvZXOg
3,17234,0.053,EFEnoticias,"Cataluña vuelve a entrar en la zona de riesgo alto de rebrote de la pandemia, con un EPG por encima de los 100 puntos. Se descarta recuperar las restricciones de aforo y horarios aunque sí ampliar el uso del certificado covid. https://t.co/Z2aY8v42HD"
4,17815,0.062,EFEnoticias,"El presidente de la Xunta, Alberto Núñez Feijóo, ha avanzado este jueves que la comunidad estudia volver a pedir el certificado covid en la hostelería y también en otras actividades que se celebren en interiores. https://t.co/QI3Hg5vIxG"
5,24265,0.067,dw_espanol,"""Mi #boda no se realizará"", confirmó #JacindaArdern a periodistas el sábado, después de anunciar las nuevas #restricciones contra el #Coronavirus que incluyen un tope de 100 personas con vacunación completa en los eventos. (gs) https://t.co/b3gTYyUM0F"
6,6114,0.068,EFEnoticias,"Diecinueve municipios del Cantabria han escalado este martes al nivel 3 de alerta covid (alto), lo que obligará al cierre del interior de sus establecimientos de hostelería a partir del jueves. https://t.co/EcDB3VmJuu"
7,9060,0.083,MarioBeteta,"""Todos estamos con la idea de que la inflación va a ser temporal y pasajera. A mi me preocupa que la recuperación es totalmente dispareja a causa del COVID y no sabemos si todos se van a recuperar al mismo tiempo, viene la escasez de productos... """


In [15]:
display(@htl "<h1>Ejemplos de búsqueda (mensajes aleatorios)</h1>")

for i in 1:3
    for qid in rand(1:length(D.text))
        search_and_display(index, D.text[qid], 7, D, E, T)
    end
end

<h1>Ejemplos de búsqueda (mensajes aleatorios)</h1>

  0.558570 seconds (166.64 k allocations: 3.814 MiB)


1,28909,0.0,Reforma,"La dramaturga Estela Leñero 'libera' a cinco mujeres presas del machismo; sucede en ""Mujeres en fuga"", una serie de radiomonólogos que se transmiten a partir de este lunes en @radioUANL. https://t.co/Ka3D4LZU4d"
2,26641,0.066,Milenio,"▶️ 'El Mijis' cumple 14 días desaparecido; aunque hay algunas pistas, siguen sin tener certeza de dónde podría estar En #LaMañanera, la activista Frida Guerrera leyó una carta al Presidente enviada por la esposa del exdiputado 📺 La historia con @azucenau #AzucenaALas10 https://t.co/EeIx7ST67S"
3,29367,0.066,Milenio,"#PolíticaCero | Si una celebridad como Sasha, con todas las herramientas monetarias y legales, guarda silencio durante 35 años, pensemos en mujeres que sufrieron la misma situación pero desprovistas de información y apoyos 🎙 La columna de @jairocalixto https://t.co/TzUAcbNUC2"
4,34187,0.067,azucenau,"#Relevante | La @FiscaliaCDMX lleva a proceso a tres mujeres, por su posible participación en robo agravado calificado y daño a la propiedad doloso en pandilla. Son las mujeres que fueron detenidas tras la recuperación del inmueble de la #CNDH https://t.co/sk29DlBFHl"
5,29201,0.068,azucenau,"#AzucenaALas10 | En #Atlixco, #Puebla, 10 personas fueron ejecutadas, siete hombres y tres mujeres, presuntamente, según la fiscalía del estado, se trató a un ajuste de cuentas entre narcomenudistas https://t.co/uhMFqLewPW"
6,28618,0.068,El_Universal_Mx,"Las autoridades capitalinas instalaron vallas metálicas frente a la Catedral Metropolitana, localizada a un costado de Palacio Nacional, debido a la próxima marcha que se realizará con motivo del Día Internacional de las Mujeres #Video @Alberx666 https://t.co/0XERzbTfdu"
7,6151,0.068,AdriDelgadoRuiz,"No se pierdan la entrevista #Exclusiva que vamos a tener en #ElDedoEnLaLlaga con @LorenaCuellar, gobernadora electa de #Tlaxcala, quien nos hablará de sus primeras acciones de gobierno con perspectiva de género @PartidoMorenaMx 3PM por la 98.5FM #ElHerladoRadio @heraldodemexico https://t.co/NkzgEYWxsT"


  0.526811 seconds (166.64 k allocations: 3.814 MiB)


1,13894,0.0,TelemundoNews,🇨🇴 Se salvó de milagro: un soldado colombiano se lanzó desde un helicóptero y su paracaídas no se abrió https://t.co/yIMwtGi4JO
2,1098,0.077,teleSURtv,En el marco de las tensiones que vive #China 🇨🇳 con #EEUU 🇺🇸 en diversos ámbitos la Asamblea Popular Nacional #China 🇨🇳 aprobó una ley que protege a compañías nacionales de las sanciones unilaterales de gobiernos extranjeros. https://t.co/V7GkFLJzmm
3,11866,0.077,Reforma,¡Fueron una pesadilla! Marco Asensio anotó un hat-trick y Karim Benzema un doblete en la goleada del Real Madrid 6-1 ante Mallorca. El conjunto de Ancelotti se mantiene como líder del campeonato. https://t.co/4QaMZyEeCk
4,28271,0.077,UniNoticias,"🇺🇸 #SOTU22: El tuit de Biden antes de su discurso. ""Amigos, voy a dar mi primer discurso sobre el Estado de la Unión esta noche. Hablaré sobre el progreso que hemos logrado juntos durante el último año y mi visión para construir una América mejor"". Más👉https://t.co/4NA7gkcDzz"
5,33842,0.078,dw_espanol,"El 51% de la población quiere que Canadá🇨🇦 deje de ser #monarquía, mientras que 26% quiere mantenerla, según el Instituto Angus Reid Además, 50% de quienes participaron dijo que la familia real británica🇬🇧 no tiene ningún papel relevante en #Canadá (ama) https://t.co/HKdMbTXAPO"
6,30859,0.078,el_pais,"Putin se compara con J. K. Rowling ante el intento, dice, de ""cancelar"" a la cultura rusa"", pero la escritora de Harry Potter rechaza ser puesta como ejemplo por el presidente ruso de la ""cultura de la cancelación occidental"" https://t.co/VabgzYgi18 https://t.co/wxTEcBgrAc"
7,6291,0.078,teleSURtv,"El primer encuentro en materia nuclear⚛️ entre #EEUU🇺🇸y #Rusia🇷🇺 mantenido en Ginebra, #Suiza🇨🇭tuvo la finalidad de alcanzar en el futuro un ambiente de estabilidad alrededor del mundo y de reducir los riesgos, de acuerdo con fuentes de ambas cancillerías https://t.co/1SherLjw2D"


  0.561001 seconds (166.64 k allocations: 3.814 MiB)


1,22273,0.0,Reforma,.@Sergeluna_S acusó a @lorenzocordovav de mentir por afirmar que denuncias penales contra los 6 consejeros del #INE fueron por pensar diferente https://t.co/UBsAmSq9bo
2,7615,0.07,Milenio,▶ El gobierno de México finalmente se puso las pilas para bajar la violencia en el país y optó por la vía legal: puso una demanda civil contra los fabricantes de armas de Estados Unidos ¿Qué significa esto para ambos países? @paobarquet nos explica #EnCorto https://t.co/AZgJLmwyTI
3,7832,0.07,Milenio,▶ El gobierno de México finalmente se puso las pilas para bajar la violencia en el país y optó por la vía legal: puso una demanda civil contra los fabricantes de armas de Estados Unidos ¿Qué significa esto para ambos países? @paobarquet ​nos explica #EnCorto https://t.co/dakOd6NjKl
4,38079,0.07,el_pais,"🔴 Villarejo, sobre Rajoy: “Hay que controlar a Correa, tiene información del Barbas”. El comisario hizo de agente doble en el ‘caso Gürtel’ para proteger a la vez al PP y al cabecilla de la red corrupta https://t.co/ArTirqhGpm"
5,22714,0.071,el_pais,"Alemania quiere acelerar el ritmo de la vacunación de refuerzo y restringir lo más posible los contactos, pero sin cerrar bares y restaurantes. Solo podrán acceder a ellos los vacunados que tengan la tercera dosis o que presenten un test negativo del día https://t.co/Q1srBseSYG"
6,38971,0.071,SinEmbargoMX,"Morena arrasa en 4 años | El partido del Presidente logró consolidarse como la principal fuerza política de México y luego de la jornada electoral de este domingo quedó con el poder de 20 entidades, seguido del PAN que tiene bajo su control cinco estados https://t.co/61hfaW1kCJ https://t.co/ZhoVC6EMOA"
7,27750,0.071,Milenio,"▶️ Rusia inició el despliegue militar contra Ucrania que ya ha dejado más de 130 muertos durante el primer día: ¿cómo empezó todo? Talya Iscan, internacionalista, nos explica 📺 La entrevista en #MILENIOTelevisión https://t.co/CCPvCfN1JE"


# Actividades
- Reproduzca el ejercicio de este notebook, use embeddings para español, cambié los ejemplos. Se sugiere el uso de <https://ingeotec.github.io/regional-spanish-models/> donde encontrará modelos fastText regionalizados del español, pero puede usar otros embeddings.
- ¿Qué piensa de las diferencias de tamaño entre los documentos y las consultas? esto como afecta a la representación semántica.
- ¿Cuál sería el símil de bigramas y trigramas para este esquema de representación semántica? Implementelo.
- Defina $H^\textsf{pt}_d$, donde $pt$ es la probabilidad de ocurrencia de término.
- Defina $H^\textsf{TF}_d$, donde $TF$ es la frecuencia de término normalizada.
- Implemente $H^\textsf{TF}_d$ y $H^\textsf{pt}_d$ de manera secuencial y con un índice métrico. Si usa Julia considere `SimilaritySearch.jl` y si usa Python considere `faiss`.
- Reporte su notebook y anote sus soluciones a las preguntas planteadas. El reporte deberá contener un ensayo de [@KSKW2015] como introducción. Reporte los resultados de sus implementaciones, compare contra las alternativas presentadas en este reporte. Discuta sus resultados. Finalice el reporte  con reflexiones sobre el uso de nubes de puntos en lugar de bolsas de palabras tradicionales. Anoté sus conclusiones.

# Bibliografía
- [KSKW2015] Kusner, M., Sun, Y., Kolkin, N., & Weinberger, K. (2015, June). From word embeddings to document distances. In International conference on machine learning (pp. 957-966). PMLR.
- [PW2009] Pele, O., & Werman, M. (2009, September). Fast and robust earth mover's distances. In 2009 IEEE 12th international conference on computer vision (pp. 460-467). IEEE.
